# Import the necessary libraries

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

### Load the data into Pandas dataframe

In [120]:
df = pd.read_csv("../archive/combined_emotion.csv")
df.drop_duplicates(inplace = True)
df.head()

,sentence,emotion
0,i just feel really helpless and heavy hearted,fear
1,ive enjoyed being able to slouch about relax a...,sad
2,i gave up my internship with the dmrg and am f...,fear
3,i dont know i feel so lost,sad
4,i am a kindergarten teacher and i am thoroughl...,fear


Drop the columns that are not required for the neural network.

Understand the distribution better.

* Create input and output vectors.
* Process the labels.

In [145]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

X = df["sentence"]
Y = df["emotion"]


# Step 1: Convert string labels to integer labels
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)  # Converts 'joy' → 0, 'sad' → 1, etc.

# Step 2: Convert integer labels to one-hot encoding
Y = to_categorical(Y, num_classes=6)

# Print label mapping (optional)
print(dict(zip(label_encoder.classes_, range(len(label_encoder.classes_)))))


Y

{'anger': 0, 'fear': 1, 'joy': 2, 'love': 3, 'sad': 4, 'suprise': 5}


array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

Split into training and test data.

In [160]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


### Process the data
* Tokenize the data and convert the text to sequences.
* Add padding to ensure that all the sequences have the same shape.
* There are many ways of taking the *max_len* and here an arbitrary length of 150 is chosen.

In [147]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)


In [161]:
sequences_matrix

array([[  0,   0,   0, ..., 103,  85,   4],
       [  0,   0,   0, ...,  90,  13,  12],
       [  0,   0,   0, ..., 100,  58,  47],
       ...,
       [  0,   0,   0, ...,   4,   5,  15],
       [  0,   0,   0, ...,  11,  10,  78],
       [  0,   0,   0, ..., 202,   7, 505]])

### RNN
Define the RNN structure.

In [168]:
emotion_classes = df['emotion'].unique()
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(len(emotion_classes), activation='softmax', name='out_layer')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

Call the function and compile the model.

In [169]:
from tensorflow.keras.optimizers import Adam

model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])


c:\Users\medkh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_12 (Embedding)        │ (None, 150, 50)        │        50,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC1 (Dense)                     │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_14 (Activation)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out_layer (Dense)               │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,622 (381.34 KB)

 Trainable params: 97,622 (381.34 KB)

 Non-trainable params: 0 (0.00 B)

Fit on the training data.

In [167]:
Y_train

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [170]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Your label encoding
labels = np.array(['anger', 'fear', 'joy', 'love', 'sad', 'suprise'])

# Assuming Y_train is one-hot encoded
# Example Y_train (one-hot encoded) - You should have this data in your case
# Y_train.shape -> (num_samples, 6)

# Step 1: Calculate class weights for each class
# Convert the labels to numerical representation (if needed)
class_labels = np.array([0, 1, 2, 3, 4, 5])  # The integer labels for each class

# Compute class weights for each individual class in the multi-class setup
class_weights = compute_class_weight(class_weight="balanced", classes=class_labels, y=Y_train.argmax(axis=1))

# Create a dictionary of weights for each class
class_weight_dict = {class_labels[i]: class_weights[i] for i in range(len(class_weights))}

print("Class Weight Dictionary:", class_weight_dict)

# Step 2: Train your model with these class weights
model.fit(
    sequences_matrix,  # Your feature matrix
    Y_train,           # One-hot encoded labels
    batch_size=32,
    epochs=10,
    class_weight=class_weight_dict,  # This is where the class weights are applied
    validation_split=0.2
)


Class Weight Dictionary: {0: 1.2117631642169175, 1: 1.4566290364925176, 2: 0.4929237111203902, 3: 2.0049506739421097, 4: 0.572769129124169, 5: 4.654223638956464}
Epoch 1/10
8323/8323 ━━━━━━━━━━━━━━━━━━━━ 243s 29ms/step - accuracy: 0.4087 - loss: 1.3574 - val_accuracy: 0.8258 - val_loss: 0.5895
Epoch 2/10
8323/8323 ━━━━━━━━━━━━━━━━━━━━ 243s 29ms/step - accuracy: 0.8241 - loss: 0.5312 - val_accuracy: 0.8290 - val_loss: 0.5100
Epoch 3/10
8323/8323 ━━━━━━━━━━━━━━━━━━━━ 243s 29ms/step - accuracy: 0.8329 - loss: 0.4794 - val_accuracy: 0.8377 - val_loss: 0.4823
Epoch 4/10
8323/8323 ━━━━━━━━━━━━━━━━━━━━ 240s 29ms/step - accuracy: 0.8458 - loss: 0.4279 - val_accuracy: 0.8554 - val_loss: 0.4009
Epoch 5/10
8323/8323 ━━━━━━━━━━━━━━━━━━━━ 239s 29ms/step - accuracy: 0.8608 - loss: 0.3674 - val_accuracy: 0.8686 - val_loss: 0.3422
Epoch 6/10
8323/8323 ━━━━━━━━━━━━━━━━━━━━ 242s 29ms/step - accuracy: 0.8700 - loss: 0.3303 - val_accuracy: 0.8730 - val_loss: 0.3175
Epoch 7/10
8323/8323 ━━━━━━━━━━━━━━━━━━━

The model performs well on the validation set and this configuration is chosen as the final model.

Process the test set data.

In [171]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

Evaluate the model on the test set.

In [172]:
accr = model.evaluate(test_sequences_matrix,Y_test)

2601/2601 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step - accuracy: 0.8657 - loss: 0.3281


In [173]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.326
  Accuracy: 0.866


In [174]:
# Make predictions
y_pred = model.predict(test_sequences_matrix)
class_names = ['anger', 'fear', 'joy', 'love', 'sad', 'suprise']

# Convert probabilities to class labels (0 or 1)
y_pred_classes = (y_pred > 0.5).astype(int)  # Use a threshold of 0.5

# Evaluate the model
print("Accuracy:", accuracy_score(Y_test, y_pred_classes))
print("\nClassification Report:\n", classification_report(Y_test, y_pred_classes, target_names=class_names))


2601/2601 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step
Accuracy: 0.8454070291378792

Classification Report:
               precision    recall  f1-score   support

       anger       0.91      0.82      0.86     11448
        fear       0.89      0.78      0.83      9574
         joy       0.99      0.82      0.90     28220
        love       0.73      0.93      0.82      6824
         sad       0.99      0.88      0.93     24121
     suprise       0.71      0.93      0.81      3038

   micro avg       0.92      0.85      0.88     83225
   macro avg       0.87      0.86      0.86     83225
weighted avg       0.94      0.85      0.88     83225
 samples avg       0.85      0.85      0.85     83225



c:\Users\medkh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
